In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statistics as stat
%matplotlib inline
df = pd.read_csv('Data.csv')

In [9]:
df

,source_id,ra,ra_error,dec,dec_error,pmra,pmra_error,pmdec,pmdec_error,parallax,...,lum_flame_lower,lum_flame_upper,evolstage_flame,flags_flame,mh_gspspec,mh_gspspec_lower,mh_gspspec_upper,teff_gspphot,teff_gspspec,ruwe
0,5937146156231203072,251.589977,0.013666,-50.638635,0.009253,-49.358441,0.017024,-84.672770,0.012676,13.398257,...,0.037522,0.047570,175,0,-1.23,-1.35,-1.15,3730.1030,3242.0,0.951887
1,5937183024214130304,250.185079,0.017953,-51.477674,0.013427,11.454068,0.028157,131.332927,0.019740,19.588654,...,6.492579,6.535044,207,0,NaN,NaN,NaN,7051.1920,NaN,0.782458
2,5937187009944533120,250.772902,0.016432,-51.598985,0.011883,6.066631,0.020651,-40.466666,0.015378,10.326266,...,0.058325,0.068466,245,0,NaN,NaN,NaN,3796.8080,NaN,1.033412
3,5937097124879929728,251.575427,0.018512,-51.192078,0.014025,42.895040,0.022741,18.180479,0.018143,7.811969,...,0.403095,0.414746,142,0,0.10,0.07,0.13,5187.4610,5161.0,0.956578
4,2268279092100963712,287.104664,0.010533,75.019613,0.010555,-27.839644,0.013564,-72.589666,0.014473,7.132954,...,0.884218,0.895562,296,0,0.35,0.33,0.36,5648.5337,5581.0,1.026369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113031,6030013931992633344,256.843330,0.017730,-28.319041,0.011389,-41.521917,0.022446,-56.631524,0.015372,18.965919,...,0.246750,0.258286,388,0,NaN,NaN,NaN,4592.8467,NaN,0.862961
113032,5961465394912723840,265.042224,0.014341,-38.590837,0.009632,-22.920695,0.017815,-27.801322,0.009220,6.977559,...,0.777196,0.792012,108,0,-0.22,-0.28,-0.18,5709.8438,5663.0,0.747447
113033,5961511952366039680,266.053857,0.014914,-38.276240,0.014761,-23.646626,0.018240,-19.299974,0.013298,6.723702,...,3.940146,3.993264,222,0,NaN,NaN,NaN,6627.0254,NaN,0.703388
113034,5961456766251197312,264.659519,0.014969,-38.573784,0.012680,42.856396,0.019490,-17.744924,0.015151,9.872332,...,0.116204,0.126257,189,0,NaN,NaN,NaN,4254.4253,NaN,0.876084


In [10]:
df=df.sort_values(by=['mass_flame_upper'])
df=df.reset_index()

In [11]:
dis=1.0/df['parallax']
year=float(365.2422)
parcs= 30856775814913673.0
au= 1.495978707*pow(10.0,11)
mm= au*0.001/(year*24.0*3600.0) 
df["pmra_kms"]=df["pmra"]*dis*mm
df["pmra_kms_E"]=df["pmra_error"]*dis*mm
df["pmdec_kms"]=df["pmdec"]*dis*mm
df["pmdec_kms_E"]=df["pmdec_error"]*dis*mm
df['vy']= +np.cos(df['ra'])*df['pmra_kms'] +np.sin(df['ra'])*(-np.sin(df['dec'])*df['pmdec_kms'] +np.cos(df['dec'])*df['radial_velocity']) 
df['vx']= -np.sin(df['ra'])*df['pmra_kms'] +np.cos(df['ra'])*(-np.sin(df['dec'])*df['pmdec_kms'] +np.cos(df['dec'])*df['radial_velocity']) 
df['vz']= np.cos(df['dec'])*df['pmdec_kms']+np.sin(df['dec'])*df['radial_velocity'] 
df['VR']= -0.0548755604162154*df['vx'] -0.8734370902348850 *df['vy'] -0.4838350155487132 *df['vz'] ### U
df['VT']= +0.4941094278755837*df['vx'] -0.4448296299600112 *df['vy'] +0.746982244497218  *df['vz'] ### V
df['VZ']= -0.8676661490190047*df['vx'] -0.1980763734312015 *df['vy'] +0.4559837761750669 *df['vz']
df['vy_E']= +np.cos(df['ra_error'])*df['pmra_kms_E'] +np.sin(df['ra_error'])*(-np.sin(df['dec_error'])*df['pmdec_kms_E'] +np.cos(df['dec_error'])*df['radial_velocity_error']) 
df['vx_E']= -np.sin(df['ra_error'])*df['pmra_kms_E'] +np.cos(df['ra_error'])*(-np.sin(df['dec_error'])*df['pmdec_kms_E'] +np.cos(df['dec_error'])*df['radial_velocity_error']) 
df['vz_E']= np.cos(df['dec_error'])*df['pmdec_kms_E']+np.sin(df['dec_error'])*df['radial_velocity_error'] 
df['VR_E']= -0.0548755604162154*df['vx_E'] -0.8734370902348850 *df['vy_E'] -0.4838350155487132 *df['vz_E'] ### U
df['VT_E']= +0.4941094278755837*df['vx_E'] -0.4448296299600112 *df['vy_E'] +0.746982244497218  *df['vz_E'] ### V
df['VZ_E']= -0.8676661490190047*df['vx_E'] -0.1980763734312015 *df['vy_E'] +0.4559837761750669 *df['vz_E']
df['V']=np.sqrt(df['VR']**2+df['VT']**2+df['VZ']**2)
df['V']=np.sqrt(df['VT']**2+df['VZ']**2)

In [12]:
X=[]
for i in range(0,len(df),150):
    if i >= 150:
        X.append(df.loc[i-150:i, :])

In [13]:
X2=[]
Y=[]
for i in range(len(X)):
    X2.append([sum(X[i]['mass_flame_upper'])/len(X[i]['mass_flame_upper']),sum(X[i]['age_flame_upper'])/len(X[i]['age_flame_upper'])])
  #  Y.append(stat.stdev(X[i]['V']))  
    X[i]['V']=X[i]['V'].sort_values()
    q3, q1 = np.percentile(X[i]['V'], [75 ,25])
    Y.append(q3 - q1)

<ipython-input-13-3680a5806b41>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i]['V']=X[i]['V'].sort_values()


In [14]:
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn import linear_model, tree, ensemble
kf =KFold(n_splits=5, shuffle=True, random_state=42)

cnt = 1
# split()  method generate indices to split data into training and test set.
for train_index, test_index in kf.split(X2, Y):
    print(f'Fold:{cnt}, Train set: {len(train_index)}, Test set:{len(test_index)}')
    cnt += 1

Fold:1, Train set: 602, Test set:151
Fold:2, Train set: 602, Test set:151
Fold:3, Train set: 602, Test set:151
Fold:4, Train set: 603, Test set:150
Fold:5, Train set: 603, Test set:150


In [18]:
Ycv=[]
def rmse(score):
    rmse = np.sqrt(-score)
    print(f'rmse= {"{:.2f}".format(rmse)}')
    Ycv.append(rmse)

In [19]:
estimators = [5,10,20,30,40,50,70,90, 100, 150, 200, 250, 300,350, 400, 450, 500]
Xcv=[]
for count in estimators:
    score = cross_val_score(ensemble.RandomForestRegressor(n_estimators= count, random_state= 42), X2, Y, cv= kf, scoring="neg_mean_squared_error")
    print(f'For estimators: {count}')
    rmse(score.mean())
    Xcv.append(count)

For estimators: 5
rmse= 3.45
For estimators: 10
rmse= 3.33
For estimators: 20
rmse= 3.30
For estimators: 30
rmse= 3.30
For estimators: 40
rmse= 3.29
For estimators: 50
rmse= 3.30
For estimators: 70
rmse= 3.29
For estimators: 90
rmse= 3.29
For estimators: 100
rmse= 3.29
For estimators: 150
rmse= 3.29
For estimators: 200
rmse= 3.29
For estimators: 250
rmse= 3.29
For estimators: 300
rmse= 3.29
For estimators: 350
rmse= 3.29
For estimators: 400
rmse= 3.29
For estimators: 450
rmse= 3.29
For estimators: 500
rmse= 3.29


In [17]:
from sklearn.tree import DecisionTreeRegressor
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

features=[]
targets=[]
X_train, X_test, y_train, y_test = train_test_split(X2, Y, test_size=0.2, random_state=4)


regressor = RandomForestRegressor(n_estimators=650, random_state=42)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
df3 = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred.flatten()})
df3

,Actual,Predicted
0,31.029765,31.424059
1,26.488330,29.592359
2,32.389649,29.727756
3,27.802509,27.416621
4,22.107630,29.480043
...,...,...
146,31.686276,27.744151
147,28.757468,27.724495
148,22.704818,26.431327
149,19.016432,17.920850
